# Gap-Based Prioritization with 3D Priority Matrices

This notebook demonstrates gap-based prioritization (Equation 6) with 2D and 3D visualization of improvement priorities.

## Priority Factors
1. **Performance Gap**: Target score - Current score
2. **Point Value**: Baldrige strategic importance
3. **Deployment Urgency**: Fraction of organization lacking implementation

**Equation**: $G_i = (T_i - S_i) \cdot v_i \cdot \delta_i$

In [ ]:
import sys
sys.path.append('../src')

from adli_letci_core import (
    compute_gap_priority_score,
    rank_improvement_priorities,
    ADLIIndicators,
    compute_adli_score
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

sns.set_style('whitegrid')

print("Gap prioritization module loaded!")

## Load and Prepare Data

In [ ]:
# Load data
df = pd.read_csv('../data/examples/sample_assessment_data.csv')

# Focus on Computer Science process items
cs_process = df[(df['department'] == 'Computer Science') & (df['item_type'] == 'Process')].copy()

# Compute current scores
cs_process['current_score'] = cs_process.apply(
    lambda row: compute_adli_score(
        ADLIIndicators(row['approach'], row['deployment'], row['learning'], row['integration'])
    ),
    axis=1
)

# Compute gaps
cs_process['gap'] = 100 - cs_process['current_score']
cs_process['deployment_urgency'] = 1.0 - cs_process['deployment']

# Compute priority scores
cs_process['priority_score'] = cs_process.apply(
    lambda row: compute_gap_priority_score(
        current_score=row['current_score'],
        target_score=100.0,
        point_value=row['point_value'],
        deployment_urgency=row['deployment_urgency']
    ),
    axis=1
)

# Sort by priority
cs_process = cs_process.sort_values('priority_score', ascending=False)

print(f"Analyzed {len(cs_process)} process items")
cs_process[['item_id', 'category', 'current_score', 'gap', 'point_value', 'deployment_urgency', 'priority_score']].head(10)

## 2D Priority Visualization: Gap vs Urgency

In [ ]:
# Create 2D scatter plot with bubble sizes
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(
    cs_process['gap'],
    cs_process['deployment_urgency'],
    s=cs_process['point_value'] * 10,  # Bubble size = point value
    c=cs_process['priority_score'],
    cmap='RdYlGn_r',
    alpha=0.6,
    edgecolors='black',
    linewidth=1.5
)

# Annotate top priorities
for idx, row in cs_process.head(5).iterrows():
    ax.annotate(
        row['item_id'],
        (row['gap'], row['deployment_urgency']),
        fontsize=11,
        fontweight='bold',
        ha='center',
        va='center'
    )

# Add quadrant lines
gap_median = cs_process['gap'].median()
urgency_median = cs_process['deployment_urgency'].median()

ax.axvline(gap_median, color='gray', linestyle='--', alpha=0.5, linewidth=2)
ax.axhline(urgency_median, color='gray', linestyle='--', alpha=0.5, linewidth=2)

# Quadrant labels
ax.text(gap_median * 1.5, urgency_median * 1.5, 'HIGH PRIORITY\n(Large Gap + High Urgency)',
       fontsize=10, ha='center', va='center', bbox=dict(boxstyle='round', facecolor='#ffcccc', alpha=0.3))
ax.text(gap_median * 0.5, urgency_median * 1.5, 'MODERATE\n(Small Gap + High Urgency)',
       fontsize=10, ha='center', va='center', bbox=dict(boxstyle='round', facecolor='#ffffcc', alpha=0.3))
ax.text(gap_median * 1.5, urgency_median * 0.5, 'STRATEGIC\n(Large Gap + Low Urgency)',
       fontsize=10, ha='center', va='center', bbox=dict(boxstyle='round', facecolor='#ccffcc', alpha=0.3))
ax.text(gap_median * 0.5, urgency_median * 0.5, 'LOW PRIORITY\n(Small Gap + Low Urgency)',
       fontsize=10, ha='center', va='center', bbox=dict(boxstyle='round', facecolor='#ccffcc', alpha=0.3))

ax.set_xlabel('Performance Gap (100 - Current Score)', fontsize=13)
ax.set_ylabel('Deployment Urgency (1 - Deployment)', fontsize=13)
ax.set_title('2D Priority Matrix: Gap vs Urgency\n(Bubble size = Point Value)', 
            fontsize=15, fontweight='bold')

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Priority Score', fontsize=12)

ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 3D Priority Visualization

In [ ]:
# Create 3D scatter plot
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

scatter_3d = ax.scatter(
    cs_process['gap'],
    cs_process['deployment_urgency'],
    cs_process['point_value'],
    c=cs_process['priority_score'],
    s=150,
    cmap='RdYlGn_r',
    alpha=0.7,
    edgecolors='black',
    linewidth=1.5
)

# Annotate top 3 priorities
for idx, row in cs_process.head(3).iterrows():
    ax.text(
        row['gap'],
        row['deployment_urgency'],
        row['point_value'],
        f"  {row['item_id']}",
        fontsize=11,
        fontweight='bold'
    )

ax.set_xlabel('Performance Gap', fontsize=12, labelpad=10)
ax.set_ylabel('Deployment Urgency', fontsize=12, labelpad=10)
ax.set_zlabel('Point Value', fontsize=12, labelpad=10)
ax.set_title('3D Priority Matrix: Gap × Urgency × Impact\n(Color = Priority Score)',
            fontsize=15, fontweight='bold', pad=20)

cbar = fig.colorbar(scatter_3d, ax=ax, shrink=0.5, aspect=10)
cbar.set_label('Priority Score', fontsize=12)

# Set viewing angle
ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

## Priority Ranking Table

In [ ]:
# Create detailed priority table
priority_table = cs_process[[
    'item_id', 'category', 'current_score', 'gap', 
    'point_value', 'deployment_urgency', 'priority_score'
]].copy()

priority_table['rank'] = range(1, len(priority_table) + 1)
priority_table = priority_table[[
    'rank', 'item_id', 'category', 'current_score', 'gap',
    'point_value', 'deployment_urgency', 'priority_score'
]]

print("\nTOP 10 IMPROVEMENT PRIORITIES")
print("=" * 100)
print(priority_table.head(10).to_string(index=False))

# Visualize top 10 priorities
fig, ax = plt.subplots(figsize=(12, 6))

top10 = priority_table.head(10)
colors_priority = ['#d62728' if i < 3 else '#ff7f0e' if i < 6 else '#1f77b4' for i in range(10)]

ax.barh(top10['item_id'], top10['priority_score'], color=colors_priority)
ax.set_xlabel('Gap Priority Score', fontsize=12)
ax.set_ylabel('Assessment Item', fontsize=12)
ax.set_title('Top 10 Improvement Priorities (Computer Science)', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add category labels
for idx, row in enumerate(top10.itertuples()):
    ax.text(row.priority_score + 20, idx, f"[{row.category}]", 
           fontsize=9, va='center', style='italic')

plt.tight_layout()
plt.show()

## Category-Level Priority Analysis

In [ ]:
# Aggregate priorities by category
category_priorities = cs_process.groupby('category').agg({
    'priority_score': ['sum', 'mean', 'max'],
    'gap': 'mean',
    'deployment_urgency': 'mean',
    'item_id': 'count'
}).round(2)

category_priorities.columns = ['Total_Priority', 'Avg_Priority', 'Max_Priority', 
                                'Avg_Gap', 'Avg_Urgency', 'Item_Count']
category_priorities = category_priorities.sort_values('Total_Priority', ascending=False)

print("\nCATEGORY-LEVEL PRIORITY ANALYSIS")
print("=" * 80)
print(category_priorities)

# Visualize category priorities
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Total priority by category
category_priorities['Total_Priority'].plot(kind='barh', ax=ax1, color='#1f77b4')
ax1.set_xlabel('Total Priority Score', fontsize=12)
ax1.set_ylabel('Category', fontsize=12)
ax1.set_title('Total Priority by Category', fontsize=13, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Scatter: Avg Gap vs Avg Urgency
for cat in category_priorities.index:
    ax2.scatter(
        category_priorities.loc[cat, 'Avg_Gap'],
        category_priorities.loc[cat, 'Avg_Urgency'],
        s=category_priorities.loc[cat, 'Total_Priority'],
        alpha=0.6,
        edgecolors='black',
        linewidth=2
    )
    ax2.annotate(cat, 
                (category_priorities.loc[cat, 'Avg_Gap'],
                 category_priorities.loc[cat, 'Avg_Urgency']),
                fontsize=10, ha='center', va='bottom')

ax2.set_xlabel('Average Performance Gap', fontsize=12)
ax2.set_ylabel('Average Deployment Urgency', fontsize=12)
ax2.set_title('Category Positioning\n(Bubble size = Total Priority)', fontsize=13, fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Multi-Department Priority Comparison

In [ ]:
# Compare priorities across departments
departments = ['Computer Science', 'Business Admin', 'Engineering']
dept_top_priorities = {}

for dept in departments:
    dept_df = df[(df['department'] == dept) & (df['item_type'] == 'Process')].copy()
    
    dept_df['current_score'] = dept_df.apply(
        lambda row: compute_adli_score(
            ADLIIndicators(row['approach'], row['deployment'], row['learning'], row['integration'])
        ),
        axis=1
    )
    
    dept_df['priority_score'] = dept_df.apply(
        lambda row: compute_gap_priority_score(
            current_score=row['current_score'],
            target_score=100.0,
            point_value=row['point_value'],
            deployment_urgency=1.0 - row['deployment']
        ),
        axis=1
    )
    
    dept_top_priorities[dept] = dept_df.nlargest(3, 'priority_score')[['item_id', 'priority_score']]

# Display comparison
print("\nTOP 3 PRIORITIES BY DEPARTMENT")
print("=" * 60)
for dept, priorities in dept_top_priorities.items():
    print(f"\n{dept}:")
    for idx, row in enumerate(priorities.itertuples(), 1):
        print(f"  {idx}. {row.item_id}: {row.priority_score:.1f}")

## Summary

### Gap-Based Prioritization Insights:

1. **Multi-Factor Assessment**: Priority considers gap, impact (point value), and urgency
2. **Visual Decision Support**: 2D/3D matrices enable intuitive priority identification
3. **Category-Level Planning**: Aggregate priorities guide resource allocation
4. **Comparative Analysis**: Cross-department comparison reveals common priorities

### Typical Priority Patterns:
- **Workforce items**: Often high priority due to deployment gaps
- **Strategy items**: Medium priority with strategic importance
- **Results items**: Lower urgency but high impact

### Action Planning:
1. Address top 3-5 priorities per cycle
2. Focus on high-urgency items with deployment gaps >0.5
3. Balance quick wins (small gaps) with strategic initiatives (high point values)

## Next Steps
- **Notebook 06**: Integration Health Index trajectory analysis
- **Notebook 07**: Scalability benchmarks